In [ ]:
import os, sys
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import spacy

nlp = spacy.load("en_core_web_sm")

print("Enter text file name: ")
fn = input().strip()

fp = os.path.join(sys.path[0], fn)

with open(fp, "r", encoding="utf-8") as f:
    content = f.read()

print("=== Original Text Sample (First 300 chars) ===\n" + content[:300] + "\n")

doc = nlp(content)

print("=== Sentences with PERSON followed by 'medal' ===")
for sent in doc.sents:
    if "medal" in sent.text.lower():
        for ent in sent.ents:
            if ent.label_ == "PERSON":
                print("- " + sent.text.strip())
                break

print("\n=== Competitions Mentioned ===")

order = [
    "the Bupa Great Ireland Run",
    "World Cross Country Championships",
    "Birmingham Grand Prix",
    "European Indoor Championships"
]

for event in order:
    if event.replace("the ", "") in content or event in content:
        print("- " + event)